In [37]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [24]:
titanic_train = pd.read_csv('Titanic_train.csv')
titanic_test = pd.read_csv('Titanic_test.csv')

In [25]:
titanic_train.isna().sum()

Unnamed: 0       0
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            143
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          552
Embarked         2
dtype: int64

In [26]:
titanic_test.isna().sum()

Unnamed: 0       0
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             34
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          135
Embarked         0
dtype: int64

In [27]:
titanic_train.shape

(712, 13)

In [28]:
# Недостающие данные по возрасту заполним средним арифметическим
mean_age = titanic_train['Age'].mean() # здесь вычисляем
print ('средний возраст пассажира: ',mean_age)
titanic_train['Age'].fillna(mean_age , inplace=True) # здесь заполняем
titanic_test['Age'].fillna(mean_age , inplace=True)

средний возраст пассажира:  29.539402460456945


In [29]:
# Больше всего пассажиров отправились в путь на Титанике из порта S - Southampton
# Назначим это значение пассажирам, чей порт посадки не известен
titanic_train['Embarked'].fillna('S',inplace=True)
titanic_test['Embarked'].fillna('S',inplace=True)

In [30]:
# удаляем лишние столбцы из датасета
del titanic_train['Unnamed: 0']
del titanic_train['PassengerId']
del titanic_train['Name']
del titanic_train['Cabin']
del titanic_train['Ticket']

del titanic_test['Unnamed: 0']
del titanic_test['PassengerId']
del titanic_test['Name']
del titanic_test['Cabin']
del titanic_test['Ticket']



In [31]:
#  сделаем предобработку данных по полу и порту посадки
# они заданы как категории, нам же для обучения нужны числа
# есть разные способы превратить категории в числа, один из них 
# - задать числовые категории по словарю

# создаем словари
dict_Sex = {'male': 1, 'female': 0}
dict_Embarked = {'S': 0, 'C': 1, 'Q':2}

# преобразуем данные при помощи map
titanic_train['Sex'] = titanic_train['Sex'].map(dict_Sex )
titanic_train['Embarked'] = titanic_train['Embarked'].map(dict_Embarked )

titanic_test['Sex'] = titanic_test['Sex'].map(dict_Sex )
titanic_test['Embarked'] = titanic_test['Embarked'].map(dict_Embarked )

In [32]:
titanic_train_per = titanic_train.drop(['Survived'], axis=1)
titanic_train_cel_per = titanic_train['Survived']

titanic_test_per = titanic_test.drop(['Survived'], axis=1)
titanic_test_cel_per = titanic_test['Survived']

In [33]:
mnb = GaussianNB()

In [34]:
mnb.fit(titanic_train_per, titanic_train_cel_per.values.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

In [35]:
# делаем предсказания на тестовой выборке
y_pred = mnb.predict(titanic_test_per)
y_pred

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0])

In [38]:
# смотрим, где угадали и не угадали
cnf_matrix = metrics.confusion_matrix(titanic_test_cel_per, y_pred)
cnf_matrix

array([[88, 18],
       [22, 51]])

In [39]:
TN = cnf_matrix[0,0] # True Negative
TP = cnf_matrix[1,1] # True Positive
FN = cnf_matrix[1,0] # False Negative
FP = cnf_matrix[0,1] # False Positive
    
Ac = mnb.score(titanic_test_per, titanic_test_cel_per)
Sens = TP/(TP+FN) 
Sp = TN/(TN+FP)
P = TP/(TP+FP)
typeI = FP/(FP+TN)
typeII = FN/(FN+TP)
    
print('Accuracy: ', Ac)
print('Sensitivity: ', Sens)
print('Specificity: ', Sp)
print('Precision: ', P)
print('Type I error rate: ', typeI)
print('Type II error rate: ', typeII)

Accuracy:  0.776536312849162
Sensitivity:  0.6986301369863014
Specificity:  0.8301886792452831
Precision:  0.7391304347826086
Type I error rate:  0.16981132075471697
Type II error rate:  0.3013698630136986
